#  Writers Room: UI Feedback Loop for Season Planner

In [ ]:
import json
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

load_dotenv()
client = OpenAI()
MEMORY_FILE = Path("../Memory/writers_room_memory.json")

In [ ]:
def init_memory():
    MEMORY_FILE.parent.mkdir(parents=True, exist_ok=True)
    if not MEMORY_FILE.exists():
        with open(MEMORY_FILE, "w") as f:
            json.dump({}, f)

def load_memory():
    init_memory()
    with open(MEMORY_FILE, "r") as f:
        return json.load(f)

def save_to_memory(table: str, key: str, content):
    init_memory()
    with open(MEMORY_FILE, "r+") as f:
        db = json.load(f)
        db.setdefault(table, {})[key] = content
        f.seek(0)
        json.dump(db, f, indent=2)
        f.truncate()

In [ ]:
def generate_season_arc(thesis, overview, episode_count):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a showrunner writing a season arc."},
            {"role": "user", "content": f"Write a {episode_count}-episode arc for this show.\n\nThesis: {thesis}\n\nOverview: {overview}"}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

def revise_season_arc(previous_output, feedback):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a showrunner revising a season arc."},
            {"role": "user", "content": f"Here's the current arc:\n\n{previous_output}\n\nPlease revise it based on this feedback:\n{feedback}"}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

In [ ]:
def ui_feedback_loop(title, generate_func, revise_func, *args):
    versions = []
    feedback_history = []

    current_output = generate_func(*args)
    versions.append(current_output)

    output_display = widgets.Output()
    feedback_input = widgets.Textarea(placeholder='Type feedback or leave blank to finish')
    revise_button = widgets.Button(description='Revise', button_style='primary')
    complete_button = widgets.Button(description='Accept & Save', button_style='success')

    def show_latest():
        with output_display:
            clear_output()
            display(Markdown(f"### {title} (Version {len(versions)})\n\n{versions[-1]}"))

    def on_revise_click(b):
        feedback = feedback_input.value.strip()
        if feedback:
            feedback_history.append(feedback)
            new_output = revise_func(versions[-1], feedback)
            versions.append(new_output)
            feedback_input.value = ''
            show_latest()

    def on_accept_click(b):
        final_version = versions[-1]
        save_to_memory("series_plan", title.lower().replace(' ', '_'), {
            "final_version": final_version,
            "feedback_history": feedback_history
        })
        with output_display:
            display(Markdown(f"✅ **Saved final version to memory under 'series_plan' → '{title.lower().replace(' ', '_')}'**"))

    revise_button.on_click(on_revise_click)
    complete_button.on_click(on_accept_click)

    show_latest()
    display(widgets.VBox([
        output_display,
        feedback_input,
        widgets.HBox([revise_button, complete_button])
    ]))

In [ ]:
# Load memory and inputs
memory = load_memory()
thesis = memory.get("concept", {}).get("show_thesis", {}).get("final_thesis", "")
overview = memory.get("series_plan", {}).get("series_overview", {}).get("final_version", "")
if not thesis or not overview:
    raise ValueError("❌ Missing show thesis or series overview. Please generate them first.")

# UI for choosing season and episode count
season_dropdown = widgets.Dropdown(
    options=[f"season_{i}" for i in range(1, 6)],
    description='Season:',
    value='season_1'
)
episode_input = widgets.BoundedIntText(value=10, min=1, max=20, description='Episodes:')
run_button = widgets.Button(description='Generate Season Arc', button_style='info')

def run_planner(b):
    clear_output()
    display(season_dropdown, episode_input, run_button)
    ui_feedback_loop(season_dropdown.value, generate_season_arc, revise_season_arc, thesis, overview, episode_input.value)

run_button.on_click(run_planner)
display(season_dropdown, episode_input, run_button)

Dropdown(description='Season:', options=('season_1', 'season_2', 'season_3', 'season_4', 'season_5'), value='s…

BoundedIntText(value=8, description='Episodes:', max=20, min=1)

Button(button_style='info', description='Generate Season Arc', style=ButtonStyle())